In [1]:
import pandas as pd
import numpy as np

# Download the data

In [2]:
mgra_based_data = pd.read_csv(r"C:\Users\cra\OneDrive - San Diego Association of Governments\Working_Drive\new_popsim\new_data\new_mgra15_based_input_2022_01.csv")
print("MGRA based data is downloaded")
synthetic_households = pd.read_csv(r"C:\Users\cra\OneDrive - San Diego Association of Governments\Working_Drive\new_popsim\new_data\synthetic_households_2022_01.csv")
print("Synthetic Households is downloaded")
synthetic_persons = pd.read_csv(r"C:\Users\cra\OneDrive - San Diego Association of Governments\Working_Drive\new_popsim\new_data\synthetic_persons_2022_01.csv")
print("Synthetic Persons is downloaded")

MGRA based data is downloaded
Synthetic Households is downloaded
Synthetic Persons is downloaded


# Check 1
Null value checks - all 

In [4]:
mgra_based_data.isna().sum()[mgra_based_data.isna().sum() > 0]

Series([], dtype: int64)

In [5]:
synthetic_households.isna().sum()[synthetic_households.isna().sum() > 0]

WIF    495495
VEH    116411
dtype: int64

In [6]:
synthetic_persons.isna().sum()[synthetic_persons.isna().sum() > 0]

Series([], dtype: int64)

In [5]:
# '''Subset the persons file using the houshold_id where WIF is NaN. Then further subset where ESR code is 1 or 4 '''
# columns = list(synthetic_households[(synthetic_households['WIF'].isna())]['household_id'])
# output = synthetic_persons[synthetic_persons['household_id'].isin(columns)]
# output = output[output['ESR'].isin([1,4])]
# output

In [6]:
# output.to_excel(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Results\SH_WIF_is_NaN_and_SP_ESR_is_1_or_4.xlsx', index=False)

# Check 2
Consistency in Household Population, Group Quarter Population, and No. Of non-GQ households between MGRA-based input, synthetic persons and synthetic households files 

In [7]:
synthetic_persons_with_GQ = synthetic_persons.merge(synthetic_households[['household_id', 'GQ_type']], how='left', on='household_id')
synthetic_persons_with_GQ

,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,SCHL,OCCP,WKW,NAICS2,SOC2,GQ_type
0,57,1,1,34,2,1.0,1.0,40.0,0,1,2,4.0,17.0,4850.0,1.0,33,41,0.0
1,57,1,2,16,2,6.0,0.0,0.0,13,1,2,0.0,13.0,0.0,0.0,0,0,0.0
2,57,1,3,15,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0.0
3,57,1,4,14,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0.0
4,57,1,5,12,1,0.0,0.0,0.0,8,1,2,0.0,8.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3282450,24280,1276879,1,41,1,6.0,0.0,0.0,0,1,1,4.0,6.0,0.0,0.0,0,0,3.0
3282451,24280,1276880,1,28,1,6.0,1.0,40.0,0,1,2,4.0,11.0,6260.0,5.0,23,47,3.0
3282452,24280,1276881,1,76,1,6.0,0.0,0.0,0,1,1,3.0,19.0,0.0,0.0,0,0,3.0
3282453,24280,1276882,1,46,1,6.0,0.0,0.0,0,9,2,4.0,13.0,0.0,0.0,0,0,3.0


In [8]:
# Synthetic Households Work 
sh_number_of_non_gq_households = len(synthetic_households[synthetic_households['GQ_type'] == 0])
sh_number_of_gq_households = len(synthetic_households[synthetic_households['GQ_type'] != 0])
sh_population_non_gq = sum(synthetic_households[synthetic_households['GQ_type'] == 0]['NP'])
sh_population_gq = sum(synthetic_households[synthetic_households['GQ_type'] != 0]['NP'])

In [9]:
# Synthetic Persons Work 
sp_number_of_non_gq_households = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0]['household_id'].nunique()
sp_number_of_gq_households = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] != 0]['household_id'].nunique()
sp_population_non_gq = len(synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0])
sp_population_gq = len(synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] != 0])
sp_population_gq

116411

In [10]:
# MGRA based information 
mb_number_of_non_gq_households = sum(mgra_based_data['hh'])
mb_population_non_gq = sum(mgra_based_data['hhp'])
mb_population_gq = sum(mgra_based_data['gq_civ'] + mgra_based_data['gq_mil'])

## Consistency in Household Population
This is non-gq

In [11]:
print(f"Synthetic Households non-gq population {sh_population_non_gq}")
print(f"Synthetic Persons non-gq population {sp_population_non_gq}")
print(f"MGRA Based non-gq population {mb_population_non_gq}")

Synthetic Households non-gq population 3166044
Synthetic Persons non-gq population 3166044
MGRA Based non-gq population 3170895


In [12]:
# Synthetic Households non-gq dataframe
sh_non_qg_pop_df = synthetic_households[synthetic_households['GQ_type'] == 0][['mgra', 'NP']]
sh_non_qg_pop_df = sh_non_qg_pop_df.groupby('mgra').sum().reset_index()
sh_non_qg_pop_df.columns = ['mgra', 'non_gq_pop_SH']

# MGRA Based non-gq dataframe
mgra_based_data_non_gq_pop = mgra_based_data[['mgra','hhp']]
mgra_based_data_non_gq_pop.columns = ['mgra', 'non_gq_pop_MB']

In [14]:
merged_sh_mb_non_gq_pop = sh_non_qg_pop_df.merge(mgra_based_data_non_gq_pop, how='left', on='mgra')
merged_sh_mb_non_gq_pop['Diff'] = merged_sh_mb_non_gq_pop['non_gq_pop_SH'] - merged_sh_mb_non_gq_pop['non_gq_pop_MB']
# merged_sh_mb_non_gq_pop.to_excel(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Data\Version 3\Results\merged_sh_mb_non_gq_pop.xlsx', index=False)
merged_sh_mb_non_gq_pop

,mgra,non_gq_pop_SH,non_gq_pop_MB,Diff
0,1,433,440,-7
1,2,63,68,-5
2,3,547,549,-2
3,4,6,5,1
4,5,93,90,3
...,...,...,...,...
18468,24314,48,46,2
18469,24316,519,514,5
18470,24317,6,5,1
18471,24318,157,136,21


In [16]:
merged_sh_mb_non_gq_pop["non_gq_pop_MB"].sum()

3170895

## Consistency in Group Quarter Population

In [17]:
print(f"Synthetic Households gq population {sh_population_gq}")
print(f"Synthetic Persons gq population {sp_population_gq}")
print(f"MGRA Based gq population {mb_population_gq}")

Synthetic Households gq population 116411
Synthetic Persons gq population 116411
MGRA Based gq population 116411


## Consistency in number of non-gq households

In [18]:
print(f"Synthetic Households -  number of non-gq households: {sh_number_of_non_gq_households}")
print(f"Synthetic Persons -  number of non-gq households: {sp_number_of_non_gq_households}")
print(f"MGRA Based -  number of non-gq households: {mb_number_of_non_gq_households}")

Synthetic Households -  number of non-gq households: 1160472
Synthetic Persons -  number of non-gq households: 1160472
MGRA Based -  number of non-gq households: 1160472


## Consistency in number of gq households

In [19]:
print(f"Synthetic Households -  number of gq households: {sh_number_of_gq_households}")
print(f"Synthetic Persons -  number of gq households: {sp_number_of_gq_households}")

Synthetic Households -  number of gq households: 116411
Synthetic Persons -  number of gq households: 116411


# Check #3
Consistency in distribution of households in income categories between synthetic households file and households' distributions sent by Econ team  

In [20]:
bins = [0, 15000, 30000, 45000, 60000, 75000, 100000, 125000, 150000, 200000, np.inf]
labels = [
    'Less than 15,000', 
    '15,000 to 30,000', 
    '30,000 to 45,000', 
    '45,000 to 60,000', 
    '60,000 to 75,000', 
    '75,000 to 100,000',
    '100,000 to 125,000',
    '125,000 to 150,000',
    '150,000 to 200,000',
    '200,000 or more'
]

synthetic_households['Income Category'] = pd.cut(synthetic_households['HHADJINC'], bins=bins, labels=labels, include_lowest=True)

In [21]:
income_frequency_distribution = synthetic_households.groupby('Income Category').size().div(len(synthetic_households)).reset_index(name='SH_percentage')
income_frequency_distribution['SH_percentage'] = round(income_frequency_distribution['SH_percentage']*100, 2)
income_frequency_distribution

,Income Category,SH_percentage
0,"Less than 15,000",9.95
1,"15,000 to 30,000",10.61
2,"30,000 to 45,000",8.90
3,"45,000 to 60,000",7.58
4,"60,000 to 75,000",7.46
5,"75,000 to 100,000",11.11
6,"100,000 to 125,000",9.68
7,"125,000 to 150,000",7.03
8,"150,000 to 200,000",10.78
9,"200,000 or more",16.90


In [17]:
income_frequency_distribution['est_percentage'] = [5.4, 8.2, 8.7, 8.6, 8.1, 12.1, 10.1, 8.3, 11.9, 18.6]
income_frequency_distribution

NameError: name 'income_frequency_distribution' is not defined

In [21]:
income_frequency_distribution['Diff'] = income_frequency_distribution['SH_percentage'] - income_frequency_distribution['est_percentage']
income_frequency_distribution

,Income Category,SH_percentage,est_percentage,Diff
0,"Less than 15,000",11.48,4.68,6.80
1,"15,000 to 30,000",11.13,7.72,3.41
2,"30,000 to 45,000",8.57,8.58,-0.01
3,"45,000 to 60,000",7.63,8.62,-0.99
4,"60,000 to 75,000",7.05,8.24,-1.19
5,"75,000 to 100,000",10.23,12.38,-2.15
6,"100,000 to 125,000",9.02,10.44,-1.42
7,"125,000 to 150,000",6.60,8.56,-1.96
8,"150,000 to 200,000",10.26,12.33,-2.07
9,"200,000 or more",18.02,18.45,-0.43


In [23]:
income_frequency_distribution#.to_excel(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-034 PopulationSim Outputs QC\Results\frequency_pct_table_SH_and_econ.xlsx', index=False)

In [ ]:
income_frequency_distribution

# Check #4
Consistency in number of workers between synthetic persons/households file and number of workers calculated by the Econ team [see production side jobs for workers]

In [33]:
sp_num_of_workers = len(synthetic_persons[(~synthetic_persons['ESR'].isna()) & (synthetic_persons['ESR'].isin([1,2,4,5]))])
sp_num_of_workers

1523764

In [34]:
econ_num = 1598.83612442623*1000
econ_num

1598836.12442623

In [35]:
sp_num_of_workers - econ_num

-75072.12442623009

In [25]:
way_1 = len(synthetic_persons[(~synthetic_persons['COW'].isna()) & (synthetic_persons['COW'] != 9)])
way_1

3230021

In [26]:
way_2 = len(synthetic_persons[(~synthetic_persons['ESR'].isna()) & (synthetic_persons['ESR'] != 6) & (synthetic_persons['ESR'] != 3)])
way_2

2178675

In [27]:
way_3 = len(synthetic_persons[(~synthetic_persons['ESR'].isna()) & (synthetic_persons['ESR'] != 6) & (synthetic_persons['ESR'] != 3) & (synthetic_persons['ESR'] != 2) & (synthetic_persons['ESR'] != 5)])
way_3

2145584

In [28]:
len(synthetic_persons[(~synthetic_persons['ESR'].isna()) & (synthetic_persons['ESR'].isin([1,4]))])

1490673

In [29]:
# This is Labor Force
len(synthetic_persons[(~synthetic_persons['ESR'].isna()) & (synthetic_persons['ESR'].isin([1,2,3]))])

1499705

In [30]:
len(synthetic_persons[(~synthetic_persons['COW'].isna()) & (synthetic_persons['COW'].isin([1,2,3,4,5]))])

1443058

In [31]:
len(synthetic_persons[(~synthetic_persons['COW'].isna()) & (synthetic_persons['COW'].isin([1,2,3,4]))])

1271556

In [ ]:
# Have a labor force check as well 
'''The control for above is from the econ team'''

In [13]:
econ_num = 1_494_959
econ_num

1494959

# Check 6

## a

In [22]:
synthetic_households['NP'].sum() == synthetic_persons['SPORDER'].count()

True

In [17]:
synthetic_persons['SPORDER'].count()

3282454

In [40]:
sh_number_of_gq_households

116411

In [35]:
print(sh_number_of_non_gq_households == sp_number_of_non_gq_households == mb_number_of_non_gq_households)

True


In [33]:
print(sh_population_gq == sp_population_gq == mb_population_gq)

True


In [25]:
mgra_based_data_non_gq_pop = mgra_based_data[['mgra','hhp']]
mgra_based_data_non_gq_pop.columns = ['mgra', 'non_gq_pop_MB']
mgra_based_data_non_gq_pop

,mgra,non_gq_pop_MB
0,1,440
1,2,68
2,3,549
3,4,5
4,5,90
...,...,...
24316,24317,5
24317,24318,136
24318,24319,0
24319,24320,0


In [11]:
sp_hh_non_gq = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0][['mgra']]
sp_hh_non_gq['sp_pop_non_gq_pop'] = 1
sp_hh_non_gq = sp_hh_non_gq.groupby('mgra').sum().reset_index()
sp_hh_non_gq

,mgra,sp_pop_non_gq_pop
0,1,441
1,2,67
2,3,547
3,4,6
4,5,90
...,...,...
18465,24314,50
18466,24316,515
18467,24317,8
18468,24318,134


In [12]:
mb_non_gq_pop = mgra_based_data[['mgra', 'hhp']]
mb_non_gq_pop.columns = ['mgra', 'mb_non_gq_pop']
mb_non_gq_pop

,mgra,mb_non_gq_pop
0,1,440
1,2,68
2,3,549
3,4,5
4,5,90
...,...,...
24316,24317,5
24317,24318,136
24318,24319,0
24319,24320,0


In [13]:
output = mb_non_gq_pop[~mb_non_gq_pop['mgra'].isin(list(sp_hh_non_gq['mgra']))]
output[output['mb_non_gq_pop'] != 0]

,mgra,mb_non_gq_pop
3307,3308,2
5021,5022,2
6649,6650,2
9886,9887,3
11115,11116,4
13622,13623,2
13869,13870,1
15803,15804,2
15959,15960,1
16220,16221,2


In [14]:
sum(mgra_based_data['hhp'])

3170895

In [15]:
output = sp_hh_non_gq.merge(mb_non_gq_pop, how='left', on='mgra')
output['diff'] = output['sp_pop_non_gq_pop'] - output['mb_non_gq_pop']

output.to_excel("mgra_non_gq_pop_differences.xlsx", index=False)

In [10]:
# Checking Equivalency 
print(sh_number_of_non_gq_households == sp_number_of_non_gq_households == mb_number_of_non_gq_households)
print(sh_population_non_gq == sp_population_non_gq == mb_population_non_gq)
print(sh_population_gq == sp_population_gq == mb_population_gq)

#TODO: num of GQ households 

True
False
True


# Check 4: Internal Consistency Checks

### Check 4a: Internal Consistency Check in aggregate values across geography levels for both persons and households outputs  

In [17]:
# Region level 
synthetic_households['NP'].sum() == len(synthetic_persons)

True

In [18]:
sh_mgra_np = synthetic_households[['mgra', 'NP']].groupby('mgra').sum().reset_index()
sh_mgra_np.columns = ['mgra', 'pop_households']
sh_mgra_np

,mgra,pop_households
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [19]:
sp_pop_count = synthetic_persons[['mgra']]
sp_pop_count['pop_persons'] = 1
sp_pop_count = sp_pop_count.groupby('mgra').sum().reset_index()
sp_pop_count

C:\Users\cra\AppData\Local\Temp\ipykernel_13544\2966902140.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_pop_count['pop_persons'] = 1


,mgra,pop_persons
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [20]:
# Check for differences at the MGRA level 
sum(sh_mgra_np['pop_households'] - sp_pop_count['pop_persons'])

0

### Check 4b: Compare total 'NP' in the households output with count of 'SPORDER' in the persons output

In [21]:
sp_mgra_sporder = synthetic_persons[['mgra', 'SPORDER']].groupby('mgra').count().reset_index()
sp_mgra_sporder.columns = ['mgra', 'sporder_count']
sp_mgra_sporder

,mgra,sporder_count
0,1,441
1,2,129
2,3,547
3,4,6
4,5,90
...,...,...
18624,24314,50
18625,24316,515
18626,24317,8
18627,24318,134


In [22]:
# Check for differences at the MGRA level 
sum(sh_mgra_np['pop_households'] - sp_mgra_sporder['sporder_count'])

0

### 4c: Compare number of households match between the households and persons output   

In [23]:
synthetic_persons['household_id'].nunique() == len(synthetic_households)

True

### 4d: Compare workers ('numworkers' and 'isWorker') from households and persons output
- We need to figure out what is the new equivalent of 'isWorker' in persons

In [24]:
sp_household_esr = synthetic_persons[['household_id', 'ESR']]

In [25]:
sh_household_wif = synthetic_households[['household_id', 'WIF']]

In [26]:
# These household IDs say that there are no workers in the family but somebody in the household has an ESR ID of [1,2,4] which means that they are a worker.  
# What is the definition os 'isworker'
output = sp_household_esr.merge(sh_household_wif, how='left', on='household_id')
output = output[output['WIF'] == 0]
output = output[~output['ESR'].isin([0,3,6])]

output

,household_id,ESR,WIF
2217,873,1.0,0.0
5444,2080,1.0,0.0
8775,3477,1.0,0.0
10124,4068,1.0,0.0
14812,5830,1.0,0.0
...,...,...,...
3165246,1160281,1.0,0.0
3165319,1160301,1.0,0.0
3165323,1160302,1.0,0.0
3165327,1160303,1.0,0.0


In [136]:
hp_cow = synthetic_persons[['household_id', 'COW']].groupby('household_id').sum().reset_index()
hp_cow

,household_id,COW
0,1,3.0
1,2,0.0
2,3,15.0
3,4,15.0
4,5,0.0
...,...,...
1276878,1276879,0.0
1276879,1276880,1.0
1276880,1276881,0.0
1276881,1276882,0.0


In [142]:
workers_comparison = hp_cow.merge(synthetic_households[['household_id', 'WIF']], how='left', on='household_id')
#workers_comparison['Diff'] = workers_comparison['COW'] - workers_comparison['WIF']

workers_comparison[((workers_comparison['COW'] != 0) & (workers_comparison['WIF'] == 0))]

,household_id,COW,WIF
110,111,1.0,0.0
119,120,4.0,0.0
128,129,5.0,0.0
129,130,5.0,0.0
154,155,1.0,0.0
...,...,...,...
1160300,1160301,10.0,0.0
1160301,1160302,10.0,0.0
1160302,1160303,10.0,0.0
1160319,1160320,1.0,0.0


# Check 5

### Check 5a: Compare number of persons by GQ and non-GQ in the persons and households output match forecast numbers.

In [108]:
print(f"Non-GQ Population - Synthetic Households - {sh_population_non_gq}")
print(f"Non-GQ Population - Synthetic Persons - {sp_population_non_gq}")
print(f"GQ Population - Synthetic Households - {sh_population_gq}")
print(f"GQ Population - Synthetic Households - {sh_population_gq}")

Non-GQ Population: Synthetic Households - 3165993


In [ ]:
print(sh_population_non_gq == sp_population_non_gq == mb_population_non_gq)
print(sh_population_gq == sp_population_gq == mb_population_gq)

# Employment Comparison

In [6]:
synthetic_persons_with_GQ = synthetic_persons.merge(synthetic_households[['household_id', 'GQ_type']], how='left', on='household_id')
sp_non_gq = synthetic_persons_with_GQ[synthetic_persons_with_GQ['GQ_type'] == 0]
sp_non_gq

,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,SCHL,OCCP,WKW,NAICS2,SOC2,xwalk_QC,GQ_type
0,57,1,1,34,2,1.0,1.0,40.0,0,1,2,4.0,17.0,4850.0,1.0,33,41,job_10,0.0
1,57,1,2,16,2,6.0,0.0,0.0,13,1,2,0.0,13.0,0.0,0.0,0,0,0,0.0
2,57,1,3,15,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0,0.0
3,57,1,4,14,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0,0.0
4,57,1,5,12,1,0.0,0.0,0.0,8,1,2,0.0,8.0,0.0,0.0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166039,10222,1160472,1,46,2,1.0,2.0,40.0,0,9,2,4.0,16.0,440.0,1.0,81,11,job_14,0.0
3166040,10222,1160472,2,49,1,1.0,1.0,40.0,0,9,2,4.0,16.0,4210.0,1.0,56,37,job_4,0.0
3166041,10222,1160472,3,16,1,6.0,0.0,0.0,12,9,2,0.0,12.0,0.0,0.0,0,0,0,0.0
3166042,10222,1160472,4,13,2,0.0,0.0,0.0,9,9,2,0.0,9.0,0.0,0.0,0,0,0,0.0


In [7]:
# Based on xwalk_PUMS_jobs.xlsx file from ENF

def PUMS_group(df):
    if df['NAICS2'] in ('92'):
        return 'job_1'
    elif df['NAICS2'] in ('MIL'):
        return 'job_2'
    elif df['NAICS2'] in ('11', '21'):
        return 'job_3'
    elif df['NAICS2'] in ('51', '54', '56'):
        return 'job_4'
    elif df['NAICS2'] in ('52', '53', '55'):
        return 'job_5'
    elif df['NAICS2'] in ('61'):
        return 'job_6'
    elif df['NAICS2'] in ('62'):
        return 'job_7'
    elif df['NAICS2'] in ('44', '45', '4M'):
        return 'job_8'
    elif df['NAICS2'] in ('48', '49', '23'):
        return 'job_9'
    elif df['NAICS2'] in ('22', '31', '32', '33', '42', '3M'):
        return 'job_10'
    elif df['NAICS2'] in ('71'):
        return 'job_11'
    elif df['NAICS2'] in ('721'):
        return 'job_12'
    elif df['NAICS2'] in ('722'):
        return 'job_13'
    elif df['NAICS2'] in ('81'):
        return 'job_14'
    else:
        return df['NAICS2']
    
sp_non_gq['xwalk_QC'] = sp_non_gq.apply(PUMS_group, axis='columns')
sp_non_gq

C:\Users\cra\AppData\Local\Temp\ipykernel_19404\2130620262.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp_non_gq['xwalk_QC'] = sp_non_gq.apply(PUMS_group, axis='columns')


,mgra,household_id,SPORDER,AGEP,SEX,ESR,COW,WKHP,SCHG,RAC1P,HISP,MIL,SCHL,OCCP,WKW,NAICS2,SOC2,xwalk_QC,GQ_type
0,57,1,1,34,2,1.0,1.0,40.0,0,1,2,4.0,17.0,4850.0,1.0,33,41,job_10,0.0
1,57,1,2,16,2,6.0,0.0,0.0,13,1,2,0.0,13.0,0.0,0.0,0,0,0,0.0
2,57,1,3,15,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0,0.0
3,57,1,4,14,2,0.0,0.0,0.0,11,1,2,0.0,11.0,0.0,0.0,0,0,0,0.0
4,57,1,5,12,1,0.0,0.0,0.0,8,1,2,0.0,8.0,0.0,0.0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166039,10222,1160472,1,46,2,1.0,2.0,40.0,0,9,2,4.0,16.0,440.0,1.0,81,11,job_14,0.0
3166040,10222,1160472,2,49,1,1.0,1.0,40.0,0,9,2,4.0,16.0,4210.0,1.0,56,37,job_4,0.0
3166041,10222,1160472,3,16,1,6.0,0.0,0.0,12,9,2,0.0,12.0,0.0,0.0,0,0,0,0.0
3166042,10222,1160472,4,13,2,0.0,0.0,0.0,9,9,2,0.0,9.0,0.0,0.0,0,0,0,0.0


In [35]:
synthetic_NAICS = sp_non_gq.groupby(['xwalk_QC']).agg(total=('NAICS2', 'count')).reset_index()

In [36]:
job_dict = {
    'job_4': 280600,
    'job_10': 177100,
    'job_7': 165200,
    'job_5': 153600,
    'job_6': 126500,
    'job_13': 124000,
    'job_9': 121500,
    'job_8': 112000,
    'job_2': 104000,
    'job_1': 98000,
    'job_14': 72300,
    'job_11': 37400,
    'job_12': 16700,
    'job_3': 9100
}


In [37]:
synthetic_NAICS['econ_values'] = synthetic_NAICS['xwalk_QC'].map(job_dict)

In [38]:
synthetic_NAICS['Diff'] = synthetic_NAICS['total'] - synthetic_NAICS['econ_values']
synthetic_NAICS.columns = ['Job_#', 'Synthetic_persons_val', 'Econ_val', 'Diff']
synthetic_NAICS = synthetic_NAICS[(synthetic_NAICS['Job_#'] != '0') & (synthetic_NAICS['Job_#'] != '99')]

In [45]:
# Extract the numeric part and create a temporary column for sorting
synthetic_NAICS['temp_sort'] = synthetic_NAICS['Job_#'].str.extract('(\d+)').astype(int)

# Sort values by the temporary column
synthetic_NAICS = synthetic_NAICS.sort_values(by='temp_sort')

# Drop the temporary column
synthetic_NAICS = synthetic_NAICS.drop(columns='temp_sort')
synthetic_NAICS

,Job_#,Synthetic_persons_val,Econ_val,Diff
2,job_1,97702,98000.0,-298.0
8,job_2,61705,104000.0,-42295.0
9,job_3,9455,9100.0,355.0
10,job_4,280734,280600.0,134.0
11,job_5,153417,153600.0,-183.0
12,job_6,125782,126500.0,-718.0
13,job_7,164773,165200.0,-427.0
14,job_8,111193,112000.0,-807.0
15,job_9,120908,121500.0,-592.0
3,job_10,176808,177100.0,-292.0


In [46]:
synthetic_NAICS.to_excel('no_gq_econ_sp_jobs_comparison.xlsx', index=False)

|